![My Image](https://raw.githubusercontent.com/ralf-42/Image/main/genai-banner-2.jpg)

<p><font size="5" color='grey'> <b>
LangChain 101
</b></font> </br></p>

---

In [1]:
#@title
#@markdown   <p><font size="4" color='green'>  Colab-Umfeld</font> </br></p>
# Installierte Python Version
import sys
print(f"Python Version: ",sys.version)
# Installierte LangChain Bibliotheken
print()
print("Installierte LangChain Bibliotheken:")

!pip list | grep '^langchain'
# Unterdrückt die "DeprecationWarning" von LangChain für die Memory-Funktionden
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning, module="langsmith.client")

Python Version:  3.11.12 (main, Apr  9 2025, 08:55:54) [GCC 11.4.0]

Installierte LangChain Bibliotheken:
langchain                             0.3.25
langchain-core                        0.3.59
langchain-text-splitters              0.3.8


In [2]:
#@title
#@markdown   <p><font size="4" color='green'>  SetUp API-Keys (setup_api_keys)</font> </br></p>
def setup_api_keys():
    """Konfiguriert alle benötigten API-Keys aus Google Colab userdata"""
    from google.colab import userdata
    import os
    from os import environ

    # Dictionary der benötigten API-Keys
    keys = {
        'OPENAI_API_KEY': 'OPENAI_API_KEY',
        'HF_TOKEN': 'HF_TOKEN',
        # Weitere Keys bei Bedarf
    }

    # Keys in Umgebungsvariablen setzen
    for env_var, key_name in keys.items():
        environ[env_var] = userdata.get(key_name)

    return {k: environ[k] for k in keys.keys()}

# Verwendung
all_keys = setup_api_keys()
# Bei Bedarf einzelne Keys direkt zugreifen
# WEATHER_API_KEY = all_keys['WEATHER_API_KEY']

# 1 | Was ist LangChain?
---

LangChain ist ein Framework zur **Entwicklung von Anwendungen mit großen Sprachmodellen** (LLMs). Es vereinfacht die Integration von LLMs in eigene Anwendungen durch:

- Modulare Komponenten für verschiedene Aufgaben
- Vorgefertigte Ketten (Chains) für komplexe Workflows
- Einfache Integration externer Datenquellen
- Werkzeuge für das Speichern von Konversationskontexten



[Einführung](https://python.langchain.com/docs/introduction/)   
[Konzepte](https://python.langchain.com/docs/concepts/)   
[API-References](https://python.langchain.com/api_reference/index.html)   
[Integrations](https://python.langchain.com/docs/integrations/providers/)

---

[Tutorials](https://python.langchain.com/docs/tutorials/)   
[How-to-Guides](https://python.langchain.com/docs/how_to/)   

In [3]:
!uv pip install --system --prerelease allow -q langchain_community langchain_openai

# 2 | Kernkonzepte
---




Diese einfache Darstellung veranschaulicht das Grundprinzip von LangChain: die Verkettung verschiedener Konzepte zu einem durchgängigen Verarbeitungsprozess, der Eingaben in strukturierte Ausgaben umwandelt.

<img src="https://raw.githubusercontent.com/ralf-42/Image/main/langchain_prozess.png" width="207" alt="Avatar">

## 2.1 Message-Typen

Beim Einsatz von Large Language Models (LLMs) wie GPT gibt es drei Hauptarten von Nachrichten (Messages), die die Interaktion mit dem Modell steuern. Diese Messages definieren, wie das Modell reagiert, welche Eingaben es erhält und welche Ausgaben es generiert.



**System Message:**   
Die `System Message` dient dazu, das Verhalten des LLMs zu steuern. Sie ist eine nicht sichtbare Nachricht, die dem Modell Anweisungen gibt, wie es sich verhalten soll. Dabei kann sie die Persönlichkeit, den Tonfall und die Einschränkungen des Modells beeinflussen. Die Systemnachricht funktioniert wie eine Art "Rollenspiel-Anweisung", um das Modell in eine gewünschte Rolle zu versetzen.

*Merkmale:*
- Definiert, wie das Modell reagieren soll
- Legt Persönlichkeit, Verhalten und Einschränkungen fest
- Wird zu Beginn einer Sitzung gesetzt und bleibt bestehen
- Nicht sichtbar für den Benutzer

*Beispiel einer System Message:*
```json
{
  "role": "system",
  "content": "Du bist ein KI-Assistent, der präzise und informative Antworten in einem professionellen Ton liefert."
}
```
Dieses Beispiel weist das Modell an, in einem professionellen Tonfall kurze und informative Antworten zu geben.

---



**User Message**   
Die `User Message` ist die eigentliche Eingabe des Benutzers. Sie kann eine Frage, eine Aufforderung, kontextrelevanten Inhalt oder ein Befehl sein. Das Modell nutzt diese Nachricht als Ausgangspunkt für die Generierung einer Antwort.

*Merkmale:*
- Direkte Eingabe des Benutzers
- Kann eine Frage, Aufforderung, kontextrelevanten Inhalt oder einen Befehl enthalten
- Basis für die Antwort des Modells

*Beispiel einer User Message:*
```json
{
  "role": "user",
  "content": "Was sind die wichtigsten Unterschiede zwischen KI und maschinellem Lernen?"
}
```
Hier fragt der Benutzer nach den Unterschieden zwischen KI und maschinellem Lernen.

---


**AI Message**   
Die `AI Message` ist die Antwort, die das Modell generiert. Sie basiert auf der User Message und den Anweisungen aus der System Message. Die AI Message kann verschiedene Eigenschaften haben, ist aber hauptsächlich auf den Inhalt fokussiert.

*Merkmale:*
- Antwort des Modells auf die Benutzeranfrage
- Kann verschiedene Eigenschaften haben (z. B. Länge, Stil)
- Hauptsächlich inhaltlich relevant

*Beispiel einer AI Message:*
```json
{
  "role": "assistant",
  "content": "Künstliche Intelligenz (KI) ist ein weites Feld, das verschiedene Teilgebiete umfasst, darunter auch das maschinelle Lernen (ML). ML konzentriert sich auf die Erstellung von Modellen, die aus Daten lernen und Vorhersagen treffen."
}
```
Hier gibt das Modell eine inhaltliche Antwort auf die Frage des Benutzers.

---



**Zusammenhang**   
Die drei Message-Typen arbeiten zusammen, um eine sinnvolle Interaktion mit dem Modell zu ermöglichen:

+  **System Message** legt die Regeln und das Verhalten des Modells fest.
+ **User Message** stellt eine Anfrage oder einen Befehl.
+ **AI Message** gibt die generierte Antwort basierend auf den vorherigen Nachrichten.

**Beispiel für eine vollständige Konversation:**    
```json
[
  {
    "role": "system",
    "content": "Sie sind ein KI-Assistent, der präzise und informative Antworten in einem professionellen Ton liefert."
  },
  {
    "role": "user",
    "content": "Kannst der erklären, was große Sprachtmodelle sind?"
  },
  {
    "role": "assistant",
    "content": "Sicher! Large Language Models (LLMs) sind fortschrittliche KI-Systeme, die mit riesigen Textdatenmengen trainiert werden. Sie können menschenähnliche Texte verstehen und generieren und sind daher für verschiedene Anwendungen wie Chatbots, Content-Erstellung und mehr nützlich."
  }
]
```
In diesem Beispiel wurde das Modell so eingestellt, dass es in einem freundlichen Tonfall antwortet, dann erhält es eine Benutzeranfrage und gibt eine passende Antwort.



**Fazit**    
Die System-, User- und AI-Message sind essenzielle Bestandteile für die Interaktion mit einem LLM. Während die System Message das Verhalten vorgibt, stellt die User Message die eigentliche Anfrage dar, auf die das Modell mit einer AI Message antwortet. Dieses Konzept ist zentral für den Einsatz von LLMs in Chatbots, KI-gestützten Assistenten und anderen interaktiven Anwendungen.


Bei einem `einfachen` Dialog kann man jedoch auf diese Trennung verzichten,



## 2.2 Prompts



Prompts sind die Eingaben bzw. Aufträge an das LLM. LangChain bietet verschiedene Template-Systeme. Ein **Template** ist eine **Vorlage** – genauer gesagt ein Text mit **Platzhaltern**, die später durch echte Werte ersetzt werden.


Ein **PromptTemplate** ist eine Vorlage für einen Eingabetext, der an ein Sprachmodell geschickt wird.
Er enthält Platzhalter ({}), die später mit Nutzereingaben oder Kontext befüllt werden.

**Templates machen die KI-Prompts:**

+ Wiederverwendbar (z. B. für viele Themen oder Fragen)

+ Strukturiert (z. B. mit Rollen und Anweisungen)

+ Dynamisch erweiterbar (mit Memory, Tools, etc.)

<p><font color='black' size="5">
Simple Prompt (ohne Rollen)
</font></p>

In [4]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

from IPython.display import display, Markdown

In [5]:
MODEL = "gpt-4o-mini"
TEMPERATUR = 0.0

In [6]:
# Modell definieren
llm = ChatOpenAI(model=MODEL, temperature=TEMPERATUR)

**Prompt wird als String übergeben:**

In [7]:
# Aufruf
response = llm.invoke("Was ist Generative KI?")

In [8]:
# Ausgabe
display(Markdown("## 📣 Model response:"))
display(Markdown("---"))
display(Markdown(response.content))

## 📣 Model response:

---

Generative KI, oder generative künstliche Intelligenz, bezieht sich auf eine Klasse von Algorithmen und Modellen, die in der Lage sind, neue Inhalte zu erzeugen. Diese Inhalte können in verschiedenen Formen vorliegen, darunter Texte, Bilder, Musik, Videos und mehr. Generative KI nutzt häufig Techniken des maschinellen Lernens, insbesondere neuronale Netzwerke, um Muster in bestehenden Daten zu erkennen und darauf basierend neue, ähnliche Daten zu generieren.

Ein bekanntes Beispiel für generative KI sind Sprachmodelle wie GPT (Generative Pre-trained Transformer), die in der Lage sind, menschenähnliche Texte zu schreiben, Fragen zu beantworten oder Geschichten zu erzählen. In der Bildgenerierung gibt es Modelle wie DALL-E oder Midjourney, die aus Textbeschreibungen Bilder erstellen können.

Generative KI hat viele Anwendungen, darunter:

1. **Kreatives Schreiben**: Unterstützung bei der Erstellung von Geschichten, Artikeln oder Gedichten.
2. **Bild- und Kunstgenerierung**: Erzeugung von Kunstwerken oder Designideen.
3. **Musikkomposition**: Erstellung neuer Musikstücke oder Melodien.
4. **Spieleentwicklung**: Generierung von Inhalten wie Levels oder Charakteren.
5. **Simulationen**: Erzeugung realistischer Daten für Trainings- und Testzwecke in verschiedenen Bereichen.

Die Technologie hat das Potenzial, kreative Prozesse zu revolutionieren, wirft jedoch auch ethische und rechtliche Fragen auf, insbesondere in Bezug auf Urheberrecht, Fälschungen und die Verbreitung von Fehlinformationen.

**Prompt wird als variabler String übergeben:**

In [9]:
# Pure Python Lösung für simple prompt mit template
thema = "Machine Learning"
prompt = f"Erkläre {thema} in einfachen Worten."

In [10]:
# Aufruf
response = llm.invoke(prompt)

In [11]:
# Ausgabe
display(Markdown("## 📣 Model response:"))
display(Markdown("---"))
display(Markdown(response.content))

## 📣 Model response:

---

Machine Learning, oder maschinelles Lernen, ist ein Teilbereich der Künstlichen Intelligenz, der es Computern ermöglicht, aus Daten zu lernen und Muster zu erkennen, ohne dass sie dafür explizit programmiert werden müssen. 

Stell dir vor, du hast viele Bilder von Katzen und Hunden. Anstatt einem Computer genau zu sagen, wie eine Katze oder ein Hund aussieht, zeigst du ihm einfach viele Beispiele. Der Computer analysiert diese Bilder und lernt, die Unterschiede zwischen Katzen und Hunden zu erkennen. 

Wenn du ihm dann ein neues Bild zeigst, kann er basierend auf dem, was er gelernt hat, vorhersagen, ob es sich um eine Katze oder einen Hund handelt. 

Zusammengefasst: Machine Learning ist wie das Lernen eines Menschen – je mehr Informationen und Erfahrungen er hat, desto besser kann er Entscheidungen treffen oder Vorhersagen machen.

**Prompt wird als template übergeben:**

In [12]:
# 1. Einfaches Template mit Platzhaltern
simple_prompt_template = PromptTemplate.from_template(
    "Erkläre {thema} in einfachen Worten."
)

In [13]:
thema = "Generative KI"
prompt = simple_prompt_template.format(thema=thema)
response = llm.invoke(prompt)

In [14]:
display(Markdown("## 📣 Model response:"))
display(Markdown("---"))
display(Markdown(response.content))

## 📣 Model response:

---

Generative KI ist eine Art von künstlicher Intelligenz, die in der Lage ist, neue Inhalte zu erstellen. Das können Texte, Bilder, Musik oder sogar Videos sein. Sie funktioniert, indem sie aus großen Mengen von Daten lernt, wie bestimmte Inhalte aussehen oder klingen. 

Stell dir vor, du hast viele Bilder von Katzen und Hunden. Eine generative KI kann dann lernen, wie diese Tiere aussehen, und neue Bilder von Katzen oder Hunden erstellen, die es so noch nicht gibt. 

Im Grunde genommen ist generative KI wie ein kreativer Computer, der neue Ideen und Inhalte basierend auf dem, was er gelernt hat, entwickeln kann.

**Vorteil PromptTemplate vs f-string**

| Kriterium                        | `f-string`                | `PromptTemplate`                         |
| -------------------------------- | ------------------------- | ---------------------------------------- |
| 🧱 Basisfunktion                 | String zusammenbauen      | String-Vorlage mit Platzhaltern          |
| 🔁 Wiederverwendbarkeit          | manuell                   | sehr gut (Template + `.format()`)        |
| 🔌 Integration mit LangChain     | ❌ Nur als fertiger Text   | ✅ Nahtlos (in Chains, mit LLMs, etc.)    |
| 📦 Kombinierbar mit Tools        | ❌ Nein                    | ✅ Ja (Memory, OutputParser, Agents)      |
| 🧠 Erweiterbar (z. B. Beispiele) | ❌ Mühsam                  | ✅ z. B. `FewShotPromptTemplate`          |
| 🕵️ Nachvollziehbarkeit          | schwer bei großen Prompts | gut dokumentierbar & testbar             |
| 👥 Rollensteuerung               | ❌ Nur manuell             | ✅ mit `ChatPromptTemplate`               |
| 🧪 Validierung                   | ❌ keine                   | ✅ Platzhalterprüfung (`input_variables`) |


<p><font color='black' size="5">
Chat Prompt (mit Rollen)
</font></p>

Das **ChatPromptTemplate** ist eine spezielle Prompt-Vorlage für Chatmodelle (z. B. GPT-3.5, GPT-4), die **mehrere Rollen und Nachrichten** unterstützt – also genau das, was Chat-Modelle eigentlich brauchen.

Es gehört zur LangChain-Bibliothek und baut auf dem Prinzip auf:


**Wer sagt was? → system, user, assistant, etc.**

In [15]:
# prompt-template als tuple
chat_template = ChatPromptTemplate([
    ("system", "Du bist ein hilfreicher und humorvoller Assistent."),
    ("user", "Erkläre mir {thema}"),
])

**... oder ...**

In [16]:
# prompt-template als dict
chat_template = ChatPromptTemplate([
    {"role": "system", "content": "Du bist ein hilfreicher und humorvoller Assistent."},
    {"role": "user", "content": "Erkläre mir {thema}"},
])

**... oder ...**

In [17]:
# prompt-template als Nachrichtenobjekte
chat_template = ChatPromptTemplate([
    SystemMessage(content="Du bist ein hilfreicher und humorvoller Assistent."),
    HumanMessage(content="Erkläre mir {thema}"),
])

In [ ]:
thema = "Machine Learning"
prompt = chat_template.format_messages(thema=thema)

In [ ]:
for message in prompt:
    print(f"Role: {message.type}, Content: {message.content}")

In [20]:
# Aufrufen der Kette mit Eingaben
response = llm.invoke(prompt)

In [21]:
display(Markdown("## 📣 Model response:"))
display(Markdown("---"))
display(Markdown(response.content))

## 📣 Model response:

---

Natürlich! Ich helfe dir gerne. Welches Thema möchtest du erklärt bekommen?

**Vorteile gegenüber `PromptTemplate`**

| Vorteil                      | `PromptTemplate`     | `ChatPromptTemplate`            |
| ---------------------------- | -------------------- | ------------------------------- |
| 🚻 Rollenstruktur            | ❌ Nur einfacher Text | ✅ Klar: `system`, `user`, `ai`  |
| 📚 Konversationsaufbau       | ❌ Nicht geeignet     | ✅ Ideal für Multi-Turn-Dialoge  |
| 🔄 Kombinierbar mit `Memory` | ❌ Nein               | ✅ Ja, via `MessagesPlaceholder` |
| 🔌 Für Chatmodelle wie GPT   | ⚠️ Nur als Fließtext | ✅ Nativ & strukturiert          |
| 🔍 Klarheit & Wartbarkeit    | ⚠️ Nur Inline-Text   | ✅ Saubere Message-Trennung      |
| 🧪 Prompt-Testbarkeit        | ❌ Eingeschränkt      | ✅ Besser strukturiert           |


<p><font color='black' size="5">
Few-Shot-Learning Prompt
</font></p>

`FewShotPromptTemplate` ist eine Prompt-Vorlage, die automatisch mehrere Beispiele einfügt, bevor das eigentliche Nutzer-Input kommt.

In [22]:
examples = [
    {"frage": "Was ist Python?", "antwort": "Eine interpretierende Programmiersprache."},
    {"frage": "Was ist Java?", "antwort": "Eine objektorientierte Sprache."}
]

In [23]:
# Few-Shot-Learning als String
few_shot_prompt = f"""
Du bist ein Assistent, der Fragen zu Programmiersprachen beantwortet. Halte deine Antworten kurz und präzise.

Beispiel 1:
Frage: {examples[0]['frage']}
Antwort: {examples[0]['antwort']}

Beispiel 2:
Frage: {examples[1]['frage']}
Antwort: {examples[1]['antwort']}

Beantworte nun die folgende Frage im gleichen Stil:
Frage: {{frage}}
Antwort:
"""

In [24]:
frage = "Was ist JavaScript?"
prompt = few_shot_prompt.format(frage=frage)

**... oder ...**

In [25]:
# Prompt für einzelne Beispiele
example_prompt = PromptTemplate.from_template(
    "Frage: {frage}\nAntwort: {antwort}"
)

In [26]:
# Few-Shot-Prompt
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="Du bist ein Assistent, der Fragen zu Programmiersprachen beantwortet. Halte deine Antworten kurz und präzise:",
    suffix="Frage: {input}\nAntwort:",
    input_variables=["input"]
)

prompt = few_shot_prompt.format(input="Was ist JavaScript?")

In [27]:
# geringere Temperaturen für mehr Sachlichkeit
llm = ChatOpenAI(model=MODEL, temperature=0.2)
response = llm.invoke(prompt)

In [28]:
display(Markdown("## 📣 Model response:"))
display(Markdown("---"))
display(Markdown(response.content))

## 📣 Model response:

---

Eine Skriptsprache für die Webentwicklung.

**Vorteile eines `FewShotPromptTemplate`**

| Vorteil                      | Beschreibung                                                      |
| ---------------------------- | ----------------------------------------------------------------- |
| 🧠 Lerneffekt für das Modell | Das Modell erkennt aus Beispielen, wie es antworten soll          |
| 🧰 Flexibilität              | Beispiele können dynamisch gesetzt oder generiert werden          |
| 🧱 Struktur & Konsistenz     | Einheitlicher Aufbau aller Beispiele über `example_prompt`        |
| 🪄 Kein Training nötig       | Kein Fine-Tuning – nur durch Prompting                            |
| 🛠 Kombinierbar              | Kann mit Templates, Tools, Chains, OutputParser kombiniert werden |


## 2.3 Modelle

LangChain unterstützt verschiedene LLMs und bietet flexible Konfigurationsmöglichkeiten:

**Beispiel `gpt-4o-mini` und `o3-mini`**

+ gpt-4o-mini ist ein kompaktes, multimodales KI-Modell von OpenAI, das Text- und Bildeingaben verarbeiten kann und besonders für schnelle, alltägliche Aufgaben sowie kleinere Anwendungen optimiert wurde

+ o3-mini ist ein spezialisiertes KI-Modell, das vor allem für komplexe Wissensverarbeitung, anspruchsvolle Denkaufgaben und professionelle Programmierprojekte entwickelt wurde

**Modellparameter:**

| **Parameter**       | **`gpt-4o-mini`**  | **`o3-mini`**  | **Erklärung**                                                                |
| ------------------- | ------------------- | ----------------- | ---------------------------------------------------------------------------- |
| `model_name`        | ✅                   | ✅                 | Modellbezeichnung z. B. `"gpt-4o-mini"` oder `"o3-mini"`                     |
| `temperature`       | ✅                   | ❌                 | Kreativität/Zufall in der Antwort (0.0 = deterministisch, 1.0 = kreativ)     |
| `top_p`             | ✅                   | ❌                 | Nucleus Sampling (alternative zu `temperature`)                              |
| `max_tokens`        | ✅                   | ✅                 | Maximale Länge der generierten Antwort                                       |
| `frequency_penalty` | ✅                   | ❌                 | Bestraft Wiederholungen im Output                                            |
| `presence_penalty`  | ✅                   | ❌                 | Fördert neue Inhalte statt Wiederholungen                                    |
| `streaming`         | ✅                   | ✅                 | Tokenweise Live-Ausgabe des Texts (Streaming API)                            |
| `seed`              | ✅ *(via kwargs)*    | ✅ *(via kwargs)*  | Fixiert Zufallselemente (Reproduzierbarkeit)                                 |
| `response_format`   | ✅ *(via kwargs)*    | ✅ *(via kwargs)*  | Format wie `"text"` oder `"json"`                                            |
| `logit_bias`        | ✅ *(via kwargs)*    | ✅ *(via kwargs)*  | Gewichtung bestimmter Tokens                                                 |
| `tool_choice`       | ✅ *(via kwargs)*    | ✅ *(via kwargs)*  | Tool-Auswahl für Function Calling                                            |
| `reasoning_effort`  | ❌                   | ✅                 | Speziell für o3-Modelle: `"low"`, `"medium"`, `"high"` – steuert Denkaufwand |


In [29]:
# Modelle konfigurieren

# Chat-Modell
chat_model = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.9
)

# Reasoning-Modell
resoning_model = ChatOpenAI(
    model_name="o3-mini",
)

In [30]:
# Gemeinsamer simple prompt
prompt = "Was ist der Unterschied zwischen Klassifikation und Regression im Machine Learning?"

In [31]:
# Aufruf
result = chat_model.invoke(prompt)

In [32]:
display(Markdown("## 📣 Chat-Model:"))
display(Markdown("---"))
display(Markdown(result.content))

## 📣 Chat-Model:

---

Klassifikation und Regression sind zwei grundlegende Arten von Aufgaben im Machine Learning, die sich in ihrem Ziel und ihrer Herangehensweise unterscheiden:

### Klassifikation
- **Ziel**: Bei der Klassifikation geht es darum, Datenpunkte in vordefinierte Kategorien oder Klassen einzuordnen. 
- **Ausgabe**: Die Ausgabe ist diskret, d.h. die Modelle liefern eine Klassenzugehörigkeit (z.B. "Ja" oder "Nein", "Hund" oder "Katze").
- **Beispiele**: 
  - E-Mail-Spam-Filter (Spam oder Nicht-Spam)
  - Bildklassifikation (Identifizierung von Objekten in Bildern)
  - Sentiment-Analyse (positiv, neutral, negativ)

### Regression
- **Ziel**: In der Regression hingegen wird versucht, eine kontinuierliche Ausgabe vorherzusagen. Es geht darum, quantitative Werte zu schätzen.
- **Ausgabe**: Die Ausgabe ist kontinuierlich, d.h. die Modelle liefern numerische Werte (z.B. Preise, Temperaturen).
- **Beispiele**:
  - Vorhersage von Hauspreisen basierend auf verschiedenen Merkmalen (z.B. Größe, Lage)
  - Wettervorhersage (Temperatur, Niederschlag)
  - Umsatzprognose für ein Unternehmen

### Zusammenfassung
- Klassifikation: Diskrete Ausgaben, Zuordnung zu Klassen.
- Regression: Kontinuierliche Ausgaben, Vorhersage von numerischen Werten.

Beide Ansätze verwenden oft ähnliche Algorithmen und Techniken, aber die Wahl zwischen Klassifikation und Regression hängt vom spezifischen Anwendungsfall und den Anforderungen an die Vorhersage ab.

In [33]:
# Aufruf
result = resoning_model.invoke(prompt)

In [50]:
display(Markdown("## 📣 Reasoning-Model:"))
display(Markdown("---"))
display(Markdown(result.content))

## 📣 Reasoning-Model:

---

AttributeError: 'dict' object has no attribute 'content'

**📊 Vergleich: Chat-Modell vs. Reasoning-Modell**

| Kriterium                 | 📣 **Chat-Modell**                                                                                             | 🧠 **Reasoning-Modell**                                                                                      |
| ------------------------- | -------------------------------------------------------------------------------------------------------------- | ------------------------------------------------------------------------------------------------------------ |
| **Stil**                  | Locker, freundlich, erklärend                                                                                  | Präzise, sachlich, eher technisch                                                                            |
| **Struktur**              | Abschnitte mit Überschriften, Beispiele, klare Gliederung in <br>*Klassifikation*, *Regression*, *Zusammenfassung* | Punktweise Gliederung mit technischer Terminologie, inkl. **Fehlermetriken** und **Entscheidungsstrategien** |
| **Zielgruppe**            | Eher auf Einsteiger\*innen oder allgemeines Publikum ausgerichtet                                              | Fachlich versierte Nutzer\*innen, z. B. Data Scientists oder Studierende in der Informatik                   |
| **Didaktik / Lesbarkeit** | Sehr zugänglich, mit einfachen Beispielen und fließendem Text                                                  | Kompakter, mehr Fachbegriffe, eher für „analytisches Nachschlagen“                                           |
| **Inhalte**               | Fokus auf Zweck und typische Beispiele                                                                         | Zusätzliche Tiefe: **Evaluationsmetriken**, **Softmax-Funktion**, **Bestimmtheitsmaß**                       |
| **Kreativität**           | Höher – wirkt wie ein Tutor, der mündlich erklärt                                                              | Geringer – wirkt wie ein Lehrbuch oder Fachaufsatz                                                           |
| **Temperatur-Effekt**     | spürbar → mehr Sprachvielfalt, z. B. beim Übergang zur Zusammenfassung                                         | gering → klare, determiniert wirkende Formulierungen                                                         |


## 2.4 Chains (LCEL)

Chains verbinden mehrere Komponenten zu einer Verarbeitungskette. LangChain Expression Language (LCEL) ist das **neue Programmiermodell in LangChain v0.3+**, das die Entwicklung von LLM-Anwendungen vereinfacht.



**Hier die Kernpunkte:**

1. Was ist LCEL?
```python
# LCEL nutzt den Pipe-Operator (|) für klare Verkettungen
chain = prompt | model | output_parser
```
- Eine deklarative Sprache zum Verketten von LangChain-Komponenten
- Ermöglicht linearen Datenfluss zwischen Komponenten
- Basiert auf dem Pipe-Operator (|) für intuitive Verbindungen

2. Warum LCEL nutzen?
- Bessere Lesbarkeit des Codes
- Einfachere Wartung und Debugging
- Verbesserte Performance durch optimierte Ausführung
- Bessere Typsicherheit und Fehlererkennung
- Unterstützt modernes Streaming und Async-Operationen

3. Praktisches Beispiel:

<p><font color='black' size="5">
Simple Chain
</font></p>

In [35]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# Prompt-Template
prompt = ChatPromptTemplate.from_messages([
    ("system", "Du bist ein hilfreicher und humorvoller Assistent."),
    ("user", "{input}")
])

#  Einfache LCEL-Kette
chain = prompt | chat_model

# Ausführung
result = chain.invoke({"input": "Erkläre LangChain Expression Language."})

In [36]:
display(Markdown("## 📣 Model response:"))
display(Markdown("---"))
display(Markdown(result.content))

## 📣 Model response:

---

LangChain Expression Language (LCEL) ist eine spezialisierte Sprache, die in der LangChain-Bibliothek verwendet wird, um komplexe logische Ausdrücke zu formulieren und zu evaluieren. LangChain selbst ist eine Framework-Umgebung, die Entwicklern hilft, mit Sprachmodellen zu arbeiten und diese in verschiedene Anwendungen zu integrieren, sei es zur Automatisierung, Datenverarbeitung oder zur Entwicklung von Chatbots.

Hier sind einige wesentliche Punkte zu LCEL:

1. **Einfachheit**: Die Sprache wurde so gestaltet, dass sie einfach zu lernen und zu verwenden ist. Auch Entwickler, die mit Programmierung nicht ganz so vertraut sind, können sie nutzen.

2. **Flexibilität**: LCEL ermöglicht es, komplexe Bedingungen und logische Ausdrücke zu formulieren, die spezifische Antworten oder Aktionen auslösen können, abhängig von den Eingabedaten.

3. **Integration**: Die Sprache ist nahtlos in die LangChain-Umgebung integriert, sodass Entwicklern die Möglichkeit geboten wird, Sprachmodelle effektiv zu steuern und deren Outputs zu verarbeiten.

4. **Beispiele**: Mit LCEL können Entwickler Bedingungen wie "Wenn die Benutzeranfrage das Wort 'Hilfe' enthält, dann antworte mit den verfügbaren Optionen" formulieren.

5. **Anwendungsfälle**: LCEL kann in verschiedenen Anwendungen eingesetzt werden, von der Bearbeitung natürlicher Sprache über die Automatisierung von Geschäftsvorgängen bis hin zur Entwicklung von interaktiven Chatbots.

Wenn du mehr über bestimmte Aspekte oder Anwendungsbeispiele von LangChain Expression Language wissen möchtest, lass es mich wissen! Vielleicht kann ich auch einen Witz über Programmiersprachen einbauen – Programmierer haben schließlich auch manchmal Schwierigkeiten, ihre „Sprache“ zu finden! 😄


LCEL ist der empfohlene Weg für alle neuen LangChain-Projekte, da es die Entwicklung vereinfacht und zukunftssicher macht.

<p><font color='black' size="5">
Sequential Chains
</font></p>

Sequentielle Chain: Linear, ein Input wird schrittweise verarbeitet

In [37]:
# Prompts
zusammenfassen_template = ChatPromptTemplate.from_messages([
    ("system", "Fasse den folgenden Text prägnant zusammen."),
    ("user", "{text}")
])

uebersetzen_template = ChatPromptTemplate.from_messages([
    ("system", "Übersetze den folgenden Text ins Deutsche."),
    ("user", "{text}")
])

In [38]:
# Verarbeitungskette

# Erste Verarbeitungskette: Zusammenfassung
summarization_chain = (
    zusammenfassen_template
    | chat_model
    | (lambda output: {"text": output})  # Notwendig, für String-> Dict, wird von translation_chain erwartet
)

# Zweite Verarbeitungskette: Translation/Übersetzung
translation_chain = (
    uebersetzen_template
    | chat_model
)

# Gesamte Kette: Zusammenfassen und dann Übersetzen
sequential_chain = summarization_chain | translation_chain

In [39]:
# Input Text
text = """
Machine Learning is a specialized branch of artificial intelligence (AI) dedicated to the development of algorithms and systems that can automatically learn from data and past experiences. Instead of following rigid, pre-defined instructions, machine learning models identify patterns, adapt their behavior, and improve their performance over time based on the information they are exposed to. This ability enables them to make predictions, recognize complex relationships, and solve problems in dynamic environments. Machine learning techniques are widely applied across various fields, including healthcare, finance, transportation, and entertainment, driving innovations such as personalized recommendations, autonomous vehicles, and intelligent diagnostics.
"""

In [40]:
# Aufruf der Kette
result = sequential_chain.invoke({"text": text})

In [41]:
display(Markdown("## 📣 Model response:"))
display(Markdown("---"))
display(Markdown(result.content))

## 📣 Model response:

---

Maschinenlernen ist ein Bereich der künstlichen Intelligenz, der Algorithmen entwickelt, die aus Daten und Erfahrungen lernen können. Anstatt festen Anweisungen zu folgen, erkennen diese Modelle Muster, passen ihr Verhalten an und verbessern sich über die Zeit. Sie finden Anwendung in vielen Bereichen wie Gesundheitswesen, Finanzen, Verkehr und Unterhaltung und ermöglichen Innovationen wie personalisierte Empfehlungen und autonome Fahrzeuge.

<p><font color='black' size="5">
Q&A-Chain
</font></p>

QA Chain: Kontextbasiert, kombiniert mehrere Inputs für eine einzelne Verarbeitung

In [42]:
# Q&A Prompt
qa_template = ChatPromptTemplate.from_messages([
    ("system", "Beantworte die Frage basierend auf dem gegebenen Kontext."),
    ("user", """
    Kontext: {context}

    Frage: {question}
    """)
])

In [43]:
# Verkettung
qa_chain = qa_template | chat_model

In [44]:
# Inhalte
context = """
Python ist eine beliebte Programmiersprache für Machine Learning.
Sie bietet viele Bibliotheken wie TensorFlow und PyTorch.
"""
question = "Welche ML-Bibliotheken gibt es für Python?"

In [45]:
# Aufruf
result = qa_chain.invoke({
    "context": context,
    "question": question
})

In [46]:
display(Markdown("## 📣 Model response:"))
display(Markdown("---"))
display(Markdown(result.content))

## 📣 Model response:

---

Für Python gibt es mehrere beliebte Bibliotheken für Machine Learning, darunter:

1. **TensorFlow** - Eine weit verbreitete Bibliothek für maschinelles Lernen und tiefes Lernen.
2. **PyTorch** - Beliebt für seine Flexibilität und Benutzerfreundlichkeit, insbesondere in der Forschung.
3. **scikit-learn** - Eine Bibliothek für traditionelle Machine Learning-Algorithmen und -Techniken.
4. **Keras** - Eine benutzerfreundliche API, die über TensorFlow aufgebaut ist und das schnelle Prototyping von Deep-Learning-Modellen ermöglicht.
5. **XGBoost** - Eine leistungsstarke Bibliothek für Gradient Boosting, die häufig für Wettbewerbe im Bereich Machine Learning verwendet wird.
6. **LightGBM** - Eine weitere Gradient-Boosting-Bibliothek, die für ihre Geschwindigkeit und Effizienz bekannt ist.
7. **CatBoost** - Eine Gradient-Boosting-Bibliothek, die speziell für kategoriale Daten entwickelt wurde.

Diese Bibliotheken bieten eine Vielzahl von Funktionen und Werkzeugen, um unterschiedliche Anforderungen im Bereich Machine Learning zu erfüllen.

## 2.5 OutputParser

Ein OutputParser ist ein Konzept in LangChain, das hilft, die Antworten eines Modells richtig weiterzuverarbeiten. Es nimmt die **rohe Ausgabe** (zum Beispiel einen langen Text) und formt sie in ein **bestimmtes Format**, das später in der Anwendung leichter benutzt werden kann.

In [47]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

# Einfacher Prompt ohne explizite Formatvorgabe
prompt = ChatPromptTemplate.from_messages([
    ("system", "Du bist ein hilfreicher Assistent. Antworte bitte im JSON-Format mit den Feldern 'name' und 'alter'."),
    ("user", "Bitte gib Name und Alter einer Person an.")
])

# Struktur der Antwort definieren
response_schemas = [
    ResponseSchema(name="name", description="Der vollständige Name der Person."),
    ResponseSchema(name="alter", description="Das Alter der Person in Jahren.")
]

# StructuredOutputParser erstellen
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)


# Modell
chat_model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# LCEL-Chain
chain = prompt | chat_model | output_parser

# Ausführen
result = chain.invoke({"input": ""})

print(result)

{'name': 'Max Mustermann', 'alter': 30}


## 2.6 Runnables


<img src="https://raw.githubusercontent.com/ralf-42/Image/main/langchain_prozess.png" width="207" alt="Avatar">

In der oben gezeigten Grafik sieht man die drei wichtigsten Runnables: Prompt, LLM und Parser, die zusammen eine Kette bilden.

Vorteile von Runnables:

+ Gleiche Bedienung: Alle Runnables (Prompt, LLM, Parser) funktionieren nach demselben Prinzip
+ Einfaches Verbinden: Man kann sie unkompliziert zu einer Kette zusammenfügen - genau wie im Bild gezeigt
+ Anpassungsfähig: Sie funktionieren sowohl einzeln als auch in der Gruppe
Schrittweise Ausgabe: Ergebnisse können stückweise weitergegeben werden

In LangChain sind die wichtigsten Komponenten als Runnables verfügbar:

+ Prompts/Templates (der erste Baustein in der Kette)
+ LLMs (das Herzstück in der Mitte)
+ Output-Parser (verwandelt die LLM-Antwort in ein nutzbares Format)
+ Komplette Chains (die ganze Verarbeitungskette)
+ Tools (für spezielle Aufgaben)
+ Retriever (holen zusätzliche Informationen)


<p><font color='black' size="5">
Benutzerdefinierte Runnables erstellen
</font></p>

Man kann auch eigene Runnables erstellen, indem man eine Klasse definiert, die das Runnable-Interface implementiert oder eine Funktion mit `RunnableLambda` wrapped:


In [92]:
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers.string import StrOutputParser

# 1. Mit RunnableLambda eine einfache Transformation erstellen
def text_verdoppeln(eingabe):
    return f"{eingabe} <br> <br>  {eingabe}"

verdoppler = RunnableLambda(text_verdoppeln)

# Verwendung des eigenen Runnables
verdoppler.invoke("Text")

'Text <br> <br>  Text'

In [93]:
# Einfacher Prompt ohne explizite Formatvorgabe
prompt = ChatPromptTemplate.from_messages([
    ("system", "Du bist ein hilfreicher Assistent. Antworte kurz."),
    ("user", "Eingabe: {eingabe}")
])

# In eine Kette einbauen - Korrektur der Pipe-Operatoren
runnable_chain_mit_verdoppler = prompt | chat_model | StrOutputParser() | verdoppler

# Test der Kette
result = runnable_chain_mit_verdoppler.invoke({"eingabe": "Wer war Einstein?"})

In [94]:
display(Markdown("## 📣 Model response:"))
display(Markdown("---"))
display(Markdown(result))

## 📣 Model response:

---

Albert Einstein war ein theoretischer Physiker, der für seine Relativitätstheorie bekannt ist. Er wurde 1879 in Deutschland geboren und gilt als einer der bedeutendsten Wissenschaftler des 20. Jahrhunderts.  <br> <br>  Albert Einstein war ein theoretischer Physiker, der für seine Relativitätstheorie bekannt ist. Er wurde 1879 in Deutschland geboren und gilt als einer der bedeutendsten Wissenschaftler des 20. Jahrhunderts. 


<p><font color='black' size="5">
Runnable-Methoden
</font></p>

Jedes Runnable unterstützt standardmäßig folgende Methoden:

1. **invoke()**: Für einzelne, synchrone Anfragen
2. **batch()**: Verarbeitet mehrere Eingaben parallel
3. **stream()**: Gibt Teilergebnisse zurück, sobald sie verfügbar sind
4. **ainvoke()**: Asynchrone Version von invoke
5. **abatch()**: Asynchrone Version von batch
6. **astream()**: Asynchrone Version von stream



Hier ein einfaches Beispiel mit den verschiedenen Aufrufmethoden:


In [95]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema import StrOutputParser

# Einfaches Template erstellen
prompt = ChatPromptTemplate.from_messages([
    ("system", "Du bist ein hilfreicher und humorvoller Assistent."),
    ("user", "{eingabe}")
])

# Modell definieren
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Output-Parser
parser = StrOutputParser()

# Runnable-Kette erstellen
runnable_chain = prompt | model | parser

In [96]:
# 1. invoke - Einzelne Anfrage
ergebnis = runnable_chain.invoke({"eingabe": "Was ist ein Runnable in LangChain?"})
print(f"INVOKE-ERGEBNIS:\n{ergebnis}\n")

INVOKE-ERGEBNIS:
In LangChain bezieht sich ein "Runnable" auf ein Konzept, das es ermöglicht, verschiedene Komponenten oder Funktionen in einer Kette von Operationen zu kombinieren und auszuführen. Es handelt sich dabei um eine abstrakte Schnittstelle, die es Entwicklern ermöglicht, ihre Logik modular und wiederverwendbar zu gestalten.

Ein Runnable kann verschiedene Aufgaben ausführen, wie z.B. Datenverarbeitung, API-Aufrufe oder die Interaktion mit Modellen. Die Idee ist, dass man verschiedene Runnables miteinander verknüpfen kann, um komplexe Workflows zu erstellen, ohne sich um die Details der Implementierung jeder einzelnen Komponente kümmern zu müssen.

Das macht es einfacher, verschiedene Teile einer Anwendung zu testen, zu debuggen und zu optimieren. Wenn du also das nächste Mal einen Runnable in LangChain siehst, denk daran: Es ist wie ein Schweizer Taschenmesser für deine Programmierlogik – vielseitig und immer bereit, dir zu helfen! 🛠️😄



In [97]:
# 2. batch - Mehrere Anfragen parallel
ergebnisse = runnable_chain.batch([
    {"eingabe": "Erkläre den Begriff LCEL kurz."},
    {"eingabe": "Was sind die Hauptvorteile von Runnables?"}
])
print("BATCH-ERGEBNISSE:")
for i, ergebnis in enumerate(ergebnisse, 1):
    print(f"Anfrage {i}: {ergebnis[:50]}...\n")

BATCH-ERGEBNISSE:
Anfrage 1: LCEL steht für "Low Carbon Energy Lab" und bezieht...

Anfrage 2: Runnables in Java sind eine praktische Möglichkeit...



In [101]:
# 3. stream - Schrittweise Ausgabe
print("STREAM-ERGEBNIS:")
for chunk in runnable_chain.stream({"eingabe": "Zähle von 1 bis 5 auf. Mache eine Pause von 3 Sekunden nach jedem Schritt."}):
    print(chunk, end="🫗")
print("\n")

STREAM-ERGEBNIS:
🫗Natürlich🫗!🫗 Hier🫗 ist🫗 die🫗 Z🫗ähl🫗ung🫗 von🫗 🫗1🫗 bis🫗 🫗5🫗 mit🫗 Pa🫗usen🫗:

🫗1🫗...🫗 (🫗Pause🫗 von🫗 🫗3🫗 Sekunden🫗)🫗  
🫗2🫗...🫗 (🫗Pause🫗 von🫗 🫗3🫗 Sekunden🫗)🫗  
🫗3🫗...🫗 (🫗Pause🫗 von🫗 🫗3🫗 Sekunden🫗)🫗  
🫗4🫗...🫗 (🫗Pause🫗 von🫗 🫗3🫗 Sekunden🫗)🫗  
🫗5🫗...🫗 

🫗Und🫗 voilà🫗!🫗 Du🫗 hast🫗 bis🫗 🫗5🫗 gez🫗ählt🫗!🫗 🎉🫗 Wenn🫗 du🫗 noch🫗 mehr🫗 zählen🫗 oder🫗 etwas🫗 anderes🫗 machen🫗 möchtest🫗,🫗 sag🫗 einfach🫗 Bes🫗cheid🫗!🫗🫗



# 3 | Überblick LangChain Konzepte
---

Konzepte von LangChain sind grundlegende Bausteine und Prinzipien, die das Framework ausmachen und seine Funktionsweise ermöglichen. Hier sind einige der wichtigsten Konzepte:

+ Chat-Modelle (**Chat models**): LLMs, die über eine Chat-API verfügbar sind und Sequenzen von Nachrichten verarbeiten1.
+ Nachrichten (**Messages**): Kommunikationseinheiten in Chat-Modellen für Ein- und Ausgabe1.
+ Chat-Verlauf (**Chat history**): Eine Sequenz von Nachrichten, die eine Konversation darstellt.
+ Tools (**Tools**): Funktionen mit definierten Schemata für Name, Beschreibung und Argumente.
+ Strukturierte Ausgabe (**Structured output**): Technik, um Chat-Modelle in strukturierten Formaten antworten zu lassen.
+ Retrieval Augmented Generation (**RAG**): Technik zur Verbesserung von Sprachmodellen durch Kombination mit externen Wissensbasen.
+ Prompt-Vorlagen (**Prompt template**s): Komponenten zur Erstellung strukturierter Prompts für LLMs.
+ Chains (**Chains**): Verknüpfungen mehrerer LLMs oder anderer Komponenten für komplexere Anwendungen.
+ Agenten (**Agents**): Nutzen Sprachmodelle, um Aktionssequenzen auszuwählen und mit externen Ressourcen zu interagieren.
+ Retriever (**Retriever**): Komponenten, die relevante Dokumente aus einer Wissensbasis abrufen.

<p><font color='black' size="5">
Anbieterbezogene APIs in LangChain
</font></p>

| API-Name in LangChain | Anbieter             | Bemerkung                                 |
|-----------------------|----------------------|-------------------------------------------|
| `ChatOpenAI`           | OpenAI               | Chat-Modelle (`gpt-3.5-turbo`, `gpt-4`, ...) |
| `OpenAI`               | OpenAI               | Text Completion (`text-davinci-003`, alt)  |
| `ChatAnthropic`        | Anthropic            | Claude-Modelle (`Claude 3`, etc.)          |
| `ChatGoogleGenerativeAI` | Google (Gemini)      | Gemini-Modelle (`gemini-1.5-pro`, etc.)    |
| `ChatCohere`           | Cohere               | Command-Modelle                           |
| `ChatMistralAI`        | Mistral              | Mistral-Modelle (`Mistral 7B`, Mixtral etc.) |
| `ChatFireworks`        | Fireworks AI         | Zugriff auf viele Open-Weight-Modelle      |
| `ChatAzureOpenAI`      | Azure (Microsoft)    | OpenAI-Modelle über Azure API              |
| `BedrockLLM`           | AWS Bedrock          | Zugang zu Claude, Titan, Jurassic, etc.    |

# A | Aufgabe
---

Die Aufgabestellungen unten bieten Anregungen, Sie können aber auch gerne eine andere Herausforderung angehen.


<p><font color='black' size="5">
Erstellen einer LangChain-Kette zur Textanalyse
</font></p>

Entwickeln Sie eine Verarbeitungskette, die einen Text analysiert und verschiedene Informationen darüber extrahiert.

**Schritte:**
1. Erstelle ein ChatPromptTemplate, das ein LLM anweist, einen Text zu analysieren
2. Die Analyse soll folgende Aspekte umfassen:
   - Hauptthema des Textes
   - Tonalität (formal, informell, etc.)
   - Schlüsselwörter (5-10)
   - Kurze Zusammenfassung (max. 3 Sätze)
3. Formatiere die Ausgabe strukturiert mit Markdown-Überschriften
4. Teste die Kette mit mindestens zwei verschiedenen Texten




<p><font color='black' size="5">
Few-Shot-Learning für Textklassifikation
</font></p>

Erstellen Sie ein System, das mithilfe von Few-Shot-Learning Texte in vorgegebene Kategorien klassifiziert.

**Schritte:**
1. Definiere 3-5 Kategorien für die Klassifikation (z.B. Sport, Politik, Technologie, Kultur)
2. Erstelle einen FewShotPromptTemplate mit Beispielen für jede Kategorie
3. Entwickle eine Chain, die neue Texte klassifiziert
4. Implementiere eine Funktion, die neben der Kategorie auch eine Begründung für die Einordnung liefert
5. Teste das System mit verschiedenen Texten, die nicht in den Beispielen vorkommen


<p><font color='black' size="5">
Q&A-System mit Sequential Chain
</font></p>

Entwickeln Sie ein System, das Fragen zu einem gegebenen Kontext beantwortet, aber zuerst den Kontext zusammenfasst und dann die Frage beantwortet.


**Schritte:**

+ Erstellen Sie zwei Templates:
    + Ein Template zur Zusammenfassung des Kontexts
    + Ein Template zur Beantwortung einer Frage basierend auf der Zusammenfassung
+ Verbinden Sie die Templates in einer sequentiellen Kette
+ Implementieren Sie einen Mechanismus, der die Zusammenfassung und die Antwort getrennt zurückgibt
+ Nutzen Sie LCEL (LangChain Expression Language) für die Verkettung
+ Testen Sie das System mit verschiedenen Kontexten und Fragen